# Exercise

In this exercise, we will learn how to use word2vec and adapt it to graph embedding.

## Word2Vec

[Word2Vec](https://papers.nips.cc/paper_files/paper/2013/hash/9aa42b31882ec039965f3c4923ce901b-Abstract.html) is a popular algorithm used for natural language processing (NLP) tasks, specifically for generating word embeddings. It was developed by researchers at Google and has gained significant attention in NLP. Word embeddings are dense vector representations of words, where words with similar meanings are represented by vectors close to each other in a high-dimensional space. Word2Vec learns these embeddings by learning from word co-occurrences.

<div>
<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*vvm6CD-3eTS-jmI0VAReZg.png" width="500"/>
</div>


### Data

We will train the word2vec using text from [Les Miserable](https://archive.org/details/lesmiserables00135gut).

In [ ]:
import requests

url = "https://ia801604.us.archive.org/27/items/lesmiserables00135gut/lesms10.txt"
response = requests.get(url)
text_data = response.text
display(text_data[:500])

word2vec takes a sequence of tokens, and each token is a word, a number, or any character. To generate the token sequences, we split the text into sentences and then split each sentence into words.

In [ ]:
from gensim import utils
import nltk

# Download the punkt tokenizer
nltk.download("punkt")

# Split the text into sentences
sentences = nltk.sent_tokenize(text_data)

# Convert sentences to words
sentences = [utils.simple_preprocess(s) for s in sentences]

In [ ]:
sentences[:2]

Next, let's prepare the word2vec model. But the question is ``which word2vec?'' It is often underappreciated that [the original paper of word2vec](https://papers.nips.cc/paper_files/paper/2013/hash/9aa42b31882ec039965f3c4923ce901b-Abstract.html) proposed four variants of word2vec models based on the architectures and training methods.

- **Architecture: SBOW vs Skip-gram**:
    There are two main architectures of Word2Vec: Continuous Bag of Words (CBOW) and Skip-gram. CBOW predicts the target word based on its context words, while Skip-gram predicts the context words given a target word. For example, consider the following sentence,
    ```
    > "The quick brown fox jumps over the lazy dog".
    ```
    CBOW trains a neural network to predict `fox` by taking the words it accompanies as input (e.g., {`brown`, `jumps`, `over`}). Skip-gram takes `fox` as input and predicts the words it accompanies.

- **Training methods: Hierarchical soft-max vs. Skip-gram negative sampling**: Training word2vec has a critical computational challenge, and two heuristics are employed to overcome it. The choice is critical as it will result in substantially different embeddings. See [here](https://papers.nips.cc/paper_files/paper/2013/hash/9aa42b31882ec039965f3c4923ce901b-Abstract.html) and  [here](https://proceedings.neurips.cc/paper/2021/hash/ca9541826e97c4530b07dda2eba0e013-Abstract.html) for details.

Here, we will use word2vec with Skip-gram negative sampling implemented in `gensim` package.

In [ ]:
import gensim

# Build the word2vec model
w2v = gensim.models.Word2Vec(
    sentences=sentences,  # input data
    vector_size=128,  # size of the vectors
    window=5,  # window size
    min_count=2,  # minimum count of words
    epochs=3,  # number of iterations
    hs=0,  # Turn off hierarchical softmax and use negative sampling
    sg=1,  # Use skip-gram instead of CBOW
)

The generated vectors can be retrieved by `w2v.wv`

In [ ]:
w2v.wv["jean"]

Let's visualize the embedding of characters. Because the original embedding is high dimensional, we will project it into 2D using PCA.

In [ ]:
# Get characters' embedding
import numpy as np

les_miserables_characters = [
    "myriel",
    "napoleon",
    "mllebaptistine",
    "mmemagloire",
    "countessdelo",
    "geborand",
    "champtercier",
    "cravatte",
    "labarre",
    "jean",
    "marguerite",
    "mmeder",
    "isabeau",
    "gervais",
    "tholomyes",
    "listolier",
    "fameuil",
    "blacheville",
    "favourite",
    "dahlia",
    "zephine",
    "fantine",
    "mmethenardier",
    "thenardier",
    "cosette",
    "javert",
    "fauchelevent",
    "bamatabois",
    "perpetue",
    "simplice",
    "scaufflaire",
    "judge",
    "champmathieu",
    "brevet",
    "chenildieu",
    "cochepaille",
    "pontmercy",
    "boulatruelle",
    "eponine",
    "anzelma",
    "motherinnocent",
    "gribier",
    "jondrette",
    "mmeburgon",
    "gavroche",
    "gillenormand",
    "magnon",
    "mllegillenormand",
    "mmepontmercy",
    "mllevaubois",
    "ltgillenormand",
    "marius",
    "baronesst",
    "mabeuf",
    "enjolras",
    "combeferre",
    "prouvaire",
    "feuilly",
    "courfeyrac",
    "bahorel",
    "bossuet",
    "joly",
    "grantaire",
    "motherplutarch",
    "gueulemer",
    "babet",
    "claquesous",
    "montparnasse",
    "toussaint",
    "brujon",
    "mmehucheloup",
]
# Filter out characters that are not in the vocabulary
les_miserables_characters = [c for c in les_miserables_characters if c in w2v.wv]
# Get the embedding of the characters
emb = np.vstack([w2v.wv[c] for c in les_miserables_characters if c in w2v.wv])

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

xy = PCA(n_components=2).fit_transform(emb)

# Plot the characters
sns.set_style("white")
sns.set(font_scale=1)
sns.set_style("ticks")
plt.figure(figsize=(6, 6))
sns.scatterplot(x=xy[:, 0], y=xy[:, 1])
for i, c in enumerate(les_miserables_characters):
    plt.annotate(c, xy[i], fontsize=8)

plt.axis("off")

Although PCA is a valuable tool, it is known to be highly sensitive to outliers. This sensitivity often leads to projections that are not useful, with only a few points deviating significantly from the majority of points.

[UMAP](https://umap-learn.readthedocs.io/en/latest/) is a non-linear dimensionality reduction method that produces a more compact projection. It can take data in non-Euclidean metric space and project it to Euclidean space. For word2vec, the embedding is not a metric space but is often considered a spherical embedding, where the angle between two vectors represents the similarity of the vectors. For this reason, often cosine similarity is adopted. With UMAP, you can specify the metric space for word embedding by using the `metric` argument. A common choice for word2vec is `cosine.`

In [ ]:
import umap

# Reduce dimensionality
reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=5, metric="cosine")
xy = reducer.fit_transform(emb)


# Plot the characters
# %%
sns.set_style("white")
sns.set(font_scale=1)
sns.set_style("ticks")
plt.figure(figsize=(6, 6))
sns.scatterplot(
    x=xy[:, 0],
    y=xy[:, 1],
)
for i, c in enumerate(les_miserables_characters):
    plt.annotate(c, xy[i], fontsize=8)

plt.axis("off")

Note that the UMAP projection may not accurately represent the data. Use it solely to generate hypotheses. See this [active discussion](https://twitter.com/lpachter/status/1431325969411821572?s=12) for details.


## Adapting word2vec for graph embedding: node2vec

word2vec can learn from a sequence of tokens. The tokens can be any entities, such as words, characters, numbers, and nodes. Thus, word2vec can generate *node embedding* if we can create a sequence of nodes from a network.

One natural way to generate node sequences is *random walks*. In a random walk process, a walker starts from a node and traverses the network by randomly selecting a neighboring node at each time step.

Let's implement a function to generate a node sequence based on a random walk from a given network and a given starting point. For the sake of simplicity, let's assume that the network is unweighted.

In [ ]:
def random_walk(A, start_node_id, walk_length):
    """Random walk on a graph.

    Parameters
    ----------
    A : scipy.sparse.csr_matrix
        Adjacency matrix of the graph.
    start_node_id : int
        Id of the starting node.
    walk_length : int
        Length of the walk.

    Returns
    -------
    visited_nodes : list
        List of visited nodes.
    """
    visited_nodes = [start_node_id]
    current_node_id = start_node_id

    for _ in range(walk_length):
        # Your code here ----------------------
        # Get the neighbors of the current node
        # Hint:
        # - A.indices and A.indptr are convenient to get the neighbors
        # - The edge weight is in A.data
        # - np.random.choice is convenient to sample a node from a neighbor with probability proportional to the edge weight

        # -------------------------------------

    return visited_nodes

Let's generate the node sequences to train word2vec. We will use the network of characters in Les Miserables.

In [ ]:
import pandas as pd
from scipy import sparse

node_table = pd.read_csv("../data/lesmiserable/nodes.csv")
edge_table = pd.read_csv("../data/lesmiserable/edges.csv")

rows, cols = edge_table["src"].values, edge_table["trg"].values
weight = edge_table["weight"].values
nrows, ncols = node_table.shape[0], node_table.shape[0]
A = sparse.csr_matrix(
    (weight, (rows, cols)),
    shape=(nrows, ncols),
)
A = A + A.T

In [ ]:
# Your code to generate a random walk. Generate 20 node sequences of length 80 for each node, totalling node_table.shape[0] * 20 sequences
n_walkers = 20
walk_length = 80

# Your code here ----------------------
# Hint:
# - Use the random_walk function
# - Use a for loop to iterate over all the nodes
# - Use another for loop to iterate over the walkers

# -------------------------------------

# Generate an embedding from word2vec
# Build the word2vec model
w2v = gensim.models.Word2Vec(
    sentences=sentences,  # input data
    vector_size=128,  # size of the vectors
    window=5,  # window size
    min_count=2,  # minimum count of words
    epochs=3,  # number of iterations
    hs=0,  # Turn off hierarchical softmax and use negative sampling
    sg=1,  # Use skip-gram instead of CBOW
)

# Retrieve the embedding vectors and pack them into 2D numpy array
emb = np.vstack([w2v.wv[c] for c in np.arange(node_table.shape[0])])

In [ ]:
# Generate the UMAP of the generated embedding

# Reduce dimensionality
reducer = umap.UMAP(n_components=2, random_state=42, min_dist=0.5, n_neighbors=30)
xy = reducer.fit_transform(emb)


# Plot the characters
# %%
sns.set_style("white")
sns.set(font_scale=0.9)
sns.set_style("ticks")
plt.figure(figsize=(6, 6))
sns.scatterplot(
    x=xy[:, 0],
    y=xy[:, 1],
)

for i in range(node_table.shape[0]):
    plt.annotate(node_table.iloc[i]["name"], xy[i], fontsize=8)
plt.axis("off")

This algorithm, which uses the skip-gram negative sampling word2vec to generate graph embedding, is known as [node2vec](https://dl.acm.org/doi/10.1145/2939672.2939754).

## DeepWalk

The idea of using word2vec in conjunction with random walks for graph embedding was initially showcased by [DeepWalk](https:// dl.acm.org/doi/10.1145/2623330.2623732). DeepWalk predates node2vec and operates within a similar framework. The primary distinction is that DeepWalk employs hierarchical softmax for training word2vec, as opposed to negative sampling.

Let's implement DeepWalk and compare the embedding. It is often convenient to wrap all related functions and variables into a function or a class. Here, we will implement DeepWalk class.

In [ ]:
class DeepWalk:
    def __init__(self, window_length=10, n_walkers=50, walk_length=80):
        """DeepWalk class

        Parameters
        ----------
        p : float
          Walk bias parameter
        q : float
          In-out parameter
        """
        self.window_length = window_length
        self.n_walks = n_walkers
        self.walk_length = walk_length
        return self

    def embed(self, A, dim):
        """Embed nodes in a graph

        Parameters:
        -----------
        A : scipy sparse matrix
          Adjacency matrix
        dim : int
          Dimension of embeddings

        Returns:
        --------
        emb : numpy array (n_nodes, dim)
          Embeddings
        """

        # Your code to generate embeddings

        return emb

Generate the embedding

In [ ]:
emb = DeepWalk().embed(A, 128)

In [ ]:
# Generate the UMAP of the generated embedding

# Reduce dimensionality
reducer = umap.UMAP(n_components=2, random_state=42, min_dist=0.5, n_neighbors=30)
xy = reducer.fit_transform(emb)

# Plot the characters
# %%
sns.set_style("white")
sns.set(font_scale=0.9)
sns.set_style("ticks")
plt.figure(figsize=(6, 6))
sns.scatterplot(x=xy[:, 0], y=xy[:, 1])

for i in range(node_table.shape[0]):
    plt.annotate(node_table.iloc[i]["name"], xy[i], fontsize=8)
plt.axis("off")